In [ ]:
from bs4 import BeautifulSoup
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import numpy as np
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
print(selenium.__version__)

In [ ]:
options = webdriver.ChromeOptions()
driver_path = '/Users/Jenny/Desktop/gwangjin/chromedriver'
driver = webdriver.Chrome(executable_path=driver_path)

### **가게 메뉴 및 정보 크롤링** ###

In [ ]:
# 동별 상세 주소 입력

#value = '광진구 화양동 13-4'

#value = '광진구 군자동 149-6'

#value = '광진구 중곡동 245-7'

# value = '광진구 긴고랑로 146'

# value = '광진구 능동로32길 65'

value = '아차산로73길 48'

# # value = '광진구 천호대로 716'

# value = '광진구 자양로30길 79'

# value = '광진구 뚝섬로 569'

# value = '광진구 뚝섬로 493'

In [ ]:
# 요기요 페이지로 이동
url = 'https://www.yogiyo.co.kr/mobile/#/'
driver.get(url)
time.sleep(2)

# 검색창 선택
search_input = driver.find_element(By.XPATH, '//*[@id="search"]/div/form/input')
search_input.click()
search_input.clear()
time.sleep(0.5)

# 검색창에 지역 값 입력
search_input.send_keys(value)
time.sleep(0.5)

# 검색 버튼 클릭
search_button = driver.find_element(By.XPATH, '//*[@id="button_search_address"]/button[2]')
search_button.click()
time.sleep(1.5)

# 최상위 검색값 클릭
click_first = driver.find_element(By.XPATH, '//*[@id="search"]/div/form/ul/li[3]/a')
click_first.click()

In [ ]:
# 스크롤 끝까지
interval = 1
prev_height = driver.execute_script("return document.body.scrollHeight")

    # 반복 수행
while True:
    # 스크롤을 가장 아래로 내림
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")

    # 페이지 로딩 대기
    time.sleep(interval)

    # 현재 문서 높이를 가져와서 저장
    curr_height = driver.execute_script("return document.body.scrollHeight")
    if curr_height == prev_height:
        break

    prev_height = curr_height


# 전체 상점 개수 추출
soup = BeautifulSoup(driver.page_source, 'lxml')

store_list = []
stores = driver.find_elements(By.CLASS_NAME, 'col-sm-6')
stores_num = len(list(set(soup.find_all('div', class_='restaurant-name ng-binding'))))
for s in stores:
    store_list.append(s)
stores_num = len(list(set(stores)))
print('stores_num : ', stores_num)

#### 1차 가게 정보 크롤링

In [ ]:
start = 0

store_data = []
end = stores_num
num = start

while num < end:
    try:
        in_store = driver.find_element(By.XPATH, f'//*[@id="content"]/div/div[4]/div/div[2]/div[{num+1}]/div/table/tbody/tr/td[2]')
        driver.execute_script("arguments[0].click();", in_store)
        time.sleep(0.5)
        soup = BeautifulSoup(driver.page_source, 'lxml')

    except (NoSuchElementException, TimeoutException):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(
            lambda driver: driver.execute_script("return document.readyState") == "complete"
        )
        continue
    
    # 주소 데이터 크롤링
    address = soup.find_all('span', class_='tc ng-binding')[3].text.strip()

    # 주소 내 '광진구' 포함 여부 판별
    if '광진구' not in address:
        driver.back() # 주소 내 '광진구'가 없으면 뒤로가기
    else: # 주소 내 '광진구'가 있으면 크롤링 진행
        store_name = soup.find('span', class_='restaurant-name ng-binding').text.strip() # 가게 이름
        openhr = soup.find_all('span', class_='tc ng-binding')[1].text.strip()           # 영업 시간
        mean_rate_div = soup.find('div', class_='restaurant-star-point')                 
        mean_rate = mean_rate_div.find('strong').text.strip()                            # 평점 값
        current_url = driver.current_url                                                 # 가게 별 URL
        menu_elements = soup.find_all('ul', class_='sub-list')                           # 메뉴 리스트
        
        for parent_element in menu_elements:
            data_rows = parent_element.find_all('div', class_='menu-name ng-binding')
            
            for row in data_rows:
                menu_name = row.text.strip()

                # 사진 존재 여부 판별
                parent_row = row.find_parent('tr')
                photo_exists = 0 if not parent_row or('ng-hide' in parent_row.get('class', [])) else 1

                # 리스트에 데이터 저장
                store_data.append({
                    'store': store_name,
                    'address': address,
                    'url': current_url, 
                    'openhr': openhr,
                    'menu': menu_name,
                    'photo_exists': photo_exists,
                    'mean_rate': mean_rate
                })
        print(f'{num}. {store_name} 크롤링 완료')
        driver.back()
        time.sleep(0.5)

    num += 1  # 다음 가게로 이동

# DataFrame으로 변환 및 중복 제거
store_df = pd.DataFrame(store_data).drop_duplicates().reset_index(drop=True)
store_df

#### 2차 중복 제거 크롤링

In [ ]:
df = pd.read_csv('dataset/store_data_0405.csv', index_col=0)
store_list = df['store'].unique().tolist()

In [ ]:
start = 0
store_data = []

end = stores_num
num = start

while num < end:
    try:
        in_store = driver.find_element(By.XPATH, f'//*[@id="content"]/div/div[4]/div/div[2]/div[{num+1}]/div/table/tbody/tr/td[2]')
        driver.execute_script("arguments[0].click();", in_store)
        time.sleep(0.5)
        soup = BeautifulSoup(driver.page_source, 'lxml')

    except (NoSuchElementException, TimeoutException):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        WebDriverWait(driver, 10).until(
            lambda driver: driver.execute_script("return document.readyState") == "complete"
        )
        continue
    
    address = soup.find_all('span', class_='tc ng-binding')[3].text.strip()

    if '광진구' not in address:
        driver.back()
    else:

        store_name = soup.find('span', class_='restaurant-name ng-binding').text.strip()
        
        # 여기서 store_name이 이미 크롤링된 리스트에 있는지 확인
        if store_name in store_list:
            print(f'{num}. {store_name}은(는) 이미 리스트에 있습니다. 크롤링을 건너뜁니다.')
            driver.back()
            time.sleep(0.5)
            num += 1
            continue

        store_name = soup.find('span', class_='restaurant-name ng-binding').text.strip()
        openhr = soup.find_all('span', class_='tc ng-binding')[1].text.strip()
        mean_rate_div = soup.find('div', class_='restaurant-star-point')
        mean_rate = mean_rate_div.find('strong').text.strip()
        current_url = driver.current_url
        menu_elements = soup.find_all('ul', class_='sub-list')
        
        for parent_element in menu_elements:
            data_rows = parent_element.find_all('div', class_='menu-name ng-binding')
            
            for row in data_rows:
                menu_name = row.text.strip()
                parent_row = row.find_parent('tr')
                photo_exists = 0 if not parent_row or ('ng-hide' in parent_row.get('class', [])) else 1

                store_data.append({
                    'store': store_name,
                    'address': address,
                    'url': current_url, 
                    'openhr': openhr,
                    'menu': menu_name,
                    'photo_exists': photo_exists,
                    'mean_rate': mean_rate
                })
        print(f'{num}. {store_name} 크롤링 완료')
        driver.back()
        time.sleep(0.5)

    num += 1  # 다음 가게로 이동

# DataFrame으로 변환 및 중복 제거
store_df = pd.DataFrame(store_data).drop_duplicates().reset_index(drop=True)
store_df

In [ ]:
store_df = pd.DataFrame(store_data).drop_duplicates().reset_index(drop=True)
store_df.to_csv('dataset/0405_자양동(2).csv')
store_df

### **리뷰 데이터 크롤링**

In [ ]:
df = pd.read_csv("dataset/store_data.csv", index_col=0)
url_list = df['url'].unique().tolist()
url_list

In [ ]:
review_data = []
failed_urls = []

for url in url_list:
    try:
        driver.get(url)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'lxml')

        # 가게 기본 데이터
        store_name = soup.find('span', class_='restaurant-name ng-binding').text.strip()
        address = soup.find_all('span', class_='tc ng-binding')[3].text.strip()

        # 리뷰 데이터
        review_btn = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')
        review_btn.click()
        time.sleep(1)

        def collect_reviews():
            soup = BeautifulSoup(driver.page_source, 'lxml')
            review_elements = soup.find_all('li', class_="list-group-item star-point ng-scope")
            for review_element in review_elements:
                score = review_element.find('span', class_='category')
                categorical_score = score.find_all('span', class_='points ng-binding')
                taste_score = categorical_score[0].text.strip()
                quantity_score = categorical_score[1].text.strip()
                review_time = review_element.find('span', class_='review-time ng-binding').text.strip()
                delivery_score = categorical_score[2].text.strip() if len(categorical_score) > 2 else None
                review_content = review_element.find('p', class_='ng-binding').text.strip()

                review_data.append({
                    'store': store_name,
                    'address': address,
                    'taste_score': taste_score,
                    'quantity_score': quantity_score,
                    'delivery_score': delivery_score,
                    'review': review_content,
                    'review_time': review_time
                })

        collect_reviews()

        # 더보기 버튼 클릭 함수
        def click_more_button():
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                plus_btn = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, plus_btn_xpath))
                )
                plus_btn.click()
                return True
            except:
                return False

        i = 0
        while True:
            i += 1
            var = i * 10 + 2
            plus_btn_xpath = f'/html/body/div[6]/div[2]/div[1]/div[5]/ul/li[{var}]/a'

            if not click_more_button():
                break

            if i % 50 == 0:
                print(f"{i*10}번째 리뷰 로딩 중...")

            time.sleep(3)

        collect_reviews()
        print(f'{store_name} 리뷰 수집 완료 ({len(review_data)}개)')

    except Exception as e:
        print(f"Error while processing {url}: {str(e)}")
        failed_urls.append(url)
        continue  # 다음 URL로 넘어갑니다.

review_df = pd.DataFrame(review_data).drop_duplicates().reset_index(drop=True)

# 실패한 URL이 있을 경우, 확인 가능
print("Failed URLs:", failed_urls)

#### **남은 가게 크롤링** ####

In [ ]:
remained_df = pd.read_csv('dataset/review_na.csv', index_col=0)
remained_list = remained_df['url'].tolist()
remained_list

In [ ]:
review_data = []
failed_urls = []

for url in remained_list:
    try:
        driver.get(url)
        time.sleep(2)

        soup = BeautifulSoup(driver.page_source, 'lxml')

        # 가게 기본 데이터
        store_name = soup.find('span', class_='restaurant-name ng-binding').text.strip()
        address = soup.find_all('span', class_='tc ng-binding')[3].text.strip()

        # 리뷰 데이터
        review_btn = driver.find_element(By.XPATH, '//*[@id="content"]/div[2]/div[1]/ul/li[2]/a')
        review_btn.click()
        time.sleep(1)

        def collect_reviews():
            soup = BeautifulSoup(driver.page_source, 'lxml')
            review_elements = soup.find_all('li', class_="list-group-item star-point ng-scope")
            for review_element in review_elements:
                score = review_element.find('span', class_='category')
                categorical_score = score.find_all('span', class_='points ng-binding')
                taste_score = categorical_score[0].text.strip()
                quantity_score = categorical_score[1].text.strip()
                review_time = review_element.find('span', class_='review-time ng-binding').text.strip()
                delivery_score = categorical_score[2].text.strip() if len(categorical_score) > 2 else None
                review_content = review_element.find('p', class_='ng-binding').text.strip()

                review_data.append({
                    'store': store_name,
                    'address': address,
                    'taste_score': taste_score,
                    'quantity_score': quantity_score,
                    'delivery_score': delivery_score,
                    'review': review_content,
                    'review_time': review_time
                })

        collect_reviews()

        # 더보기 버튼 클릭 함수
        def click_more_button():
            try:
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
                plus_btn = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, plus_btn_xpath))
                )
                plus_btn.click()
                return True
            except:
                return False

        i = 0
        while True:
            i += 1
            var = i * 10 + 2
            plus_btn_xpath = f'/html/body/div[6]/div[2]/div[1]/div[5]/ul/li[{var}]/a'

            if not click_more_button():
                break

            if i % 50 == 0:
                print(f"{i*10}번째 리뷰 로딩 중...")

            time.sleep(3)  # 클릭 후 대기 시간 추가

        collect_reviews()
        print(f'{store_name} 리뷰 수집 완료 ({len(review_data)}개)')

    except Exception as e:
        print(f"Error while processing {url}: {str(e)}")
        failed_urls.append(url)
        continue  # 다음 URL로 넘어갑니다.

review_df = pd.DataFrame(review_data).drop_duplicates().reset_index(drop=True)

# 실패한 URL이 있을 경우, 확인 가능
print("Failed URLs:", failed_urls)

In [ ]:
# csv 파일로 저장

main = pd.read_csv('dataset/review_data.csv', index_col=0)
main = pd.concat([main, review_df], axis=0).reset_index(drop=True)
main.to_csv('dataset/review_data.csv')
main